In [3]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets,layers,models,Model

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = "/content/drive/My Drive/Data/"

dataset = pd.read_csv(path + 'imdb.csv')
dataset

Mounted at /content/drive


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [16]:
import re
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D,Conv1D,LSTM
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [18]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocessing_text(text):
    # Remove html tag
    sentence = remove_tags(text)
    # Remove link
    sentence = re.sub(r'https:\/\/[a-zA-Z]*\.com',' ',sentence)
    # Remove number
    sentence = re.sub(r'\d+',' ',sentence)
    # Remove white space
    sentence = re.sub(r'\s+',' ',sentence)
    # Remove single character
    sentence = re.sub(r"\b[a-zA-Z]\b", ' ', sentence)
    # Remove bracket
    sentence = re.sub(r'\W+',' ',sentence)
    # Make sentence lowercase
    sentence = sentence.lower()
    return sentence

In [19]:
pre_proces_sen = []
sentences = list(dataset['review'])
for sen in sentences:
    pre_proces_sen.append(preprocessing_text(sen))

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [21]:
stop = ['has', 'its', "needn't", 'm', "wouldn't", 'but', 'he', "mustn't", 'his', 'there', 'or', "won't", 'can', 'd', "hadn't", 'how', 'hasn', 'very', 'wouldn', 'own', "doesn't", 'their', "isn't", 'an', "haven't", "wasn't", 'those', 'once', "shan't", 'when', "aren't", 've', 'it', "it's", 'of', "don't", 'and', 'down', 'yours', 'to', 'over', "she's", 'we', 'they', 'haven', 'having', 'ain', 'no', 'her', 'you', 'then', 'just', 'didn', 'into', 'before', 'shouldn', 'here', 'yourselves', 's', 'will', 'which', 'are', 'who', 'with', "you'd", 'this', 'me', 'themselves', "you've", 'hadn', 'mightn', 'she', 'o', 'more', 'whom', 'for', 'him', 'again', 'below', 'few', 'most', 'been', 'such', 'shan', 'is', 'ourselves', 'y', 'by', 'being', 'in', 'mustn', "you'll", 'herself', 'yourself', 'ours', 'between', 'had', 'other', "should've", 't', 'isn', 'them', 'himself', 're', 'doing', 'only', 'where', 'your', 'after', 'so', 'll', 'against', 'the', 'about', 'each', 'aren', 'wasn', "couldn't", 'have', 'ma', 'i', 'my', "mightn't", 'as', 'from', 'itself', 'under', 'same', 'why', 'any', 'our', 'be', 'off', "hasn't", 'through', "you're", 'was', 'did', "shouldn't", 'myself', 'some', 'theirs', 'hers', 'further', 'do', 'now', 'than', 'too', 'during', 'at', 'because', 'doesn', 'needn', "weren't", 'don', "didn't", 'couldn', 'what', 'does', 'if', 'up', 'on', 'these', 'should', 'all', "that'll", 'above', 'weren', 'that', 'a', 'while', 'both', 'until', 'were', 'am']

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
for i in range(len(pre_proces_sen)):
    x = pre_proces_sen[i]
    x = word_tokenize(x)
    new_x_list = [word for word in x if word not in stop]
    pre_proces_sen[i] = ' '.join(new_x_list)
    if i% 2000 == 0:
        print(i,end=" ")

0 2000 4000 6000 8000 10000 12000 14000 16000 18000 20000 22000 24000 26000 28000 30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 

In [24]:
print(pre_proces_sen[2])

thought wonderful way spend time hot summer weekend sitting air conditioned theater watching light hearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed realize not match point risk addiction thought proof woody allen still fully control style many us grown love laughed one woody comedies years dare say decade never impressed scarlet johanson managed tone sexy image jumped right average spirited young woman may not crown jewel career wittier devil wears prada interesting superman great comedy go see friends


In [25]:
y  = dataset['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [26]:
X = pre_proces_sen

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [29]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

92789


In [30]:
!brew install wget

wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

SyntaxError: ignored

In [34]:
import gensim
#WORD2VEC_MODEL = "../input/one-lac-bin/model_1_lac.bin"
#load word2vec model
word2vec = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=100000)

In [35]:
embedding_weights = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    #embedding_vector = word2vec.get(word)
    try:
        embedding_weights[index] = word2vec[word]
    except:
        pass 

In [36]:
print(word2vec['not'][:40])

[ 0.08496094 -0.09521484  0.11914062  0.11181641 -0.11132812  0.04980469
  0.11425781 -0.09863281  0.09960938 -0.04150391  0.01287842 -0.18164062
 -0.11621094  0.02380371 -0.1640625   0.12695312  0.12060547  0.00946045
  0.04150391 -0.09521484 -0.12988281 -0.11523438  0.07519531 -0.04980469
  0.06591797 -0.02905273 -0.08984375  0.08447266 -0.04785156 -0.02709961
 -0.10302734  0.11083984  0.01422119 -0.09863281  0.04125977  0.07226562
  0.10986328  0.01135254 -0.01483154  0.0456543 ]


In [37]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_weights], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(128))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [38]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 300)          27836700  
                                                                 
 lstm_1 (LSTM)               (None, 128)               219648    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 28,056,477
Trainable params: 219,777
Non-trainable params: 27,836,700
_________________________________________________________________
None


In [39]:
history =  model.fit(X_train, y_train,  batch_size=128, epochs=8, validation_split=0.2,verbose=1)

Epoch 1/8
250/250 [==============================] - 103s 404ms/step - loss: 0.4216 - acc: 0.8086 - val_loss: 0.3577 - val_acc: 0.8476
Epoch 2/8
250/250 [==============================] - 100s 401ms/step - loss: 0.3620 - acc: 0.8446 - val_loss: 0.3491 - val_acc: 0.8544
Epoch 3/8
250/250 [==============================] - 101s 403ms/step - loss: 0.3364 - acc: 0.8582 - val_loss: 0.3197 - val_acc: 0.8686
Epoch 4/8
250/250 [==============================] - 101s 403ms/step - loss: 0.3155 - acc: 0.8684 - val_loss: 0.3072 - val_acc: 0.8696
Epoch 5/8
250/250 [==============================] - 101s 403ms/step - loss: 0.2981 - acc: 0.8763 - val_loss: 0.2947 - val_acc: 0.8781
Epoch 6/8
250/250 [==============================] - 102s 406ms/step - loss: 0.2883 - acc: 0.8806 - val_loss: 0.2993 - val_acc: 0.8798
Epoch 7/8
250/250 [==============================] - 102s 408ms/step - loss: 0.2784 - acc: 0.8862 - val_loss: 0.3156 - val_acc: 0.8745
Epoch 8/8
250/250 [==============================] - 10